#### 第十章 序列的修改、散列和切片
##### 10.1 Vector类：用户定义的序列类型
##### 10.2 Vector类第一版：与Vector2d类兼容

In [3]:
# 第一版Vector类的实现代码
from array import array
import reprlib
import math
import numbers
import functools
import operator
import itertools


class Vector:
    typecode = 'd'
    shortcut_names = 'xyzt'

    def __init__(self, components):
        self._components = array(self.typecode, components)

    def __iter__(self):
        return iter(self._components)

    def __repr__(self):
        components = reprlib.repr(self._components)
        components = components[components.find('['):-1]
        return 'Vecotor({})'.format(components)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(self._components))

    def __eq__(self, other):
        return len(self) == len(othen) and all(a == b for a, b in zip(self, other))
    
    def __hash__(self):
        hashes = map(hash, self._components)
        return functools.reduce(operator.xor, hashes, 0)

    def __abs__(self):
        return math.sqrt(sum(x ** 2 for x in self))

    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(memv)

    def __len__(self):
        return len(self._components)

    def __getitem__(self, index):
        '''实现可切片'''
        cls = type(self)
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral):
            return self._components[index]
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))
    
    def __getattr__(self, name):
        '''获取属性'''
        cls = type(self)
        
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0 <= pos < len(self._components):
                return self._components[pos]
        msg = '{.__name!r} object has no attribute {!r}'
        raise AttributeError(msg.format(cls, name))
    
    def __setattr__(self, name, value):
        cls = type(self)
        if len(name) == 1:
            if name in cls.shortcut_names:
                error = 'readonly attribute {attr_name!r}'
            elif name.islower():
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else:
                error = ''
            if error:
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value)
    
    def angle(self, n):
        r = math.sqrt(sum(x * x for x in self[n:]))
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a
    
    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))
    
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'):
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)], self.angles())
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(', '.join(components))

##### 10.3 协议和鸭子类型
Python 的序列协议只需要 \_\_len\_\_ 和 \_\_getitem\_\_ 两
个方法。任何类（如 Spam），只要使用标准的签名和语义实现了这两
个方法，就能用在任何期待序列的地方。Spam 是不是哪个类的子类无
关紧要，只要提供了所需的方法即可。

##### 10.4  Vector类第2版：可切片的序列

In [29]:
v1 = Vector([3, 4, 5])
print(v1, repr(v1))
print(len(v1), v1[0], v1[-1], v1[1:])

(3.0, 4.0, 5.0) Vecotor([3.0, 4.0, 5.0])
3 3.0 5.0 (4.0, 5.0)


In [9]:
# 了解__getitem__和切片的行为
class MySeq:
    def __getitem__(self, index):
        return index

In [14]:
s = MySeq()
print(s[1])
print(s[1:4])
print(s[1:4:2])
print(s[1:4:2, 9])
print(s[1:4:2, 7:9])

1
slice(1, 4, None)
slice(1, 4, 2)
(slice(1, 4, 2), 9)
(slice(1, 4, 2), slice(7, 9, None))


In [20]:
print(slice)
print(dir(slice))
print(help(slice.indices))

<class 'slice'>
['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'indices', 'start', 'step', 'stop']
Help on method_descriptor:

indices(...)
    S.indices(len) -> (start, stop, stride)
    
    Assuming a sequence of length len, calculate the start and stop
    indices, and the stride length of the extended slice described by
    S. Out of bounds indices are clipped in a manner consistent with the
    handling of normal slices.

None


In [18]:
print(slice(None, 10, 2).indices(5))
print(slice(-3, None, None).indices(5))

(0, 5, 2)
(2, 5, 1)


##### 10.5 Vector第3版：动态存取属性
属性查找失败后，解释器会调用 \_\_getattr\_\_ 方法。简单来说，对
my\_obj.x 表达式，Python 会检查 my\_obj 实例有没有名为 x 的属性；
如果没有，到类（my\_obj.\_\_class\_\_）中查找；如果还没有，顺着继
承树继续查找。 如果依旧找不到，调用 my_obj 所属类中定义的
\_\_getattr\_\_ 方法，传入 self 和属性名称的字符串形式（如 'x'）。

In [37]:
v = Vector(range(5))
print(v)
print(v.x)
v.x = 10

(0.0, 1.0, 2.0, 3.0, 4.0)
0.0


AttributeError: readonly attribute 'x'

#### 第十一章 接口：从协议到抽象基类
##### 11.1 Python文化中的接口和协议
##### 11.2 Python喜欢序列

In [1]:
# 定义__getitem__方法，只实现序列协议的一部分，这样足够访问元素、迭代和使用in运算符了
class Foo:
    def __getitem__(self, pos):
        return range(0, 30, 10)[pos]

In [5]:
f = Foo()
print(f[1])
for i in f: print(i)
print(20 in f)
print(15 in f)

10
0
10
20
True
False


In [7]:
from collections import abc
class Struggle:
    def __len__(self): return 23
print(isinstance(Struggle(), abc.Sized))

True


##### 11.5 定义抽象基类的子类

In [13]:
from collections import namedtuple
from collections.abc import MutableSequence, Hashable

Card = namedtuple('Card', 'rank suit')


class FrenchDeck2(MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                       for rank in self.ranks]
    
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, pos):
        return self._cards[pos]
    
    def __setitem__(self, pos, value):
        self._cards[pos] = value
    
    def __delitem__(self, pos):
        del self._cards[pos]
    
    def insert(self, pos, value):
        self._cards.insert(pos, value)

In [15]:
fd2 = FrenchDeck2()
print(isinstance(fd2, Hashable), hash(fd2))

True 152713990902


##### 11.7 定义并使用一个抽象基类

In [17]:
import abc

class Tombola(abc.ABC):
    
    @abc.abstractmethod
    def load(self, iterable):
        '''从可迭代对象中添加元素'''
    
    @abc.abstractmethod
    def pick(self):
        '''随机删除元素，然后将其返回
        如果实例为空， 这个方法应该抛出'LookupError'
        '''
    
    def loaded(self):
        '''如果至少有一个元素，返回'True', 否则返回'False'
        '''
        return bool(self.inspect())
    
    def inspect(self):
        '''返回一个有序元组，有当前元素构成
        '''
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        return tuple(sorted(items))

In [18]:
# 不符合Tombola要求的子类无法蒙混过关
class Fake(Tombola):
    def pick(self):
        return 13

In [20]:
print(Fake)
f = Fake()

<class '__main__.Fake'>


TypeError: Can't instantiate abstract class Fake with abstract methods load

In [21]:
# BingoCage是Tombola的具体子类
import random

class BingoCage(Tombola):
    
    def __init__(self, items):
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)
    
    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)
    
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
    
    def __call__(self):
        self.pick()

In [24]:
class LotteryBlower(Tombola):

    def __init__(self, iterable):
        self._balls = list(iterable)

    def load(self, iterable):
        self._balls.extend(iterable)

    def pick(self):
        try:
            pos = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError('pick from empty LotteryBlower')
        return self._balls.pop(pos)

    def loaded(self):
        return bool(self._balls)

    def inspect(self):
        return tuple(sorted(self._balls))

In [25]:
# TomboList是Tombola的虚拟子类
from random import randrange

@Tombola.register
class TomboList(list):
    
    def pick(self):
        if self:
            pos = randrange(len(self))
            return self.pop(pos)
        else:
            raise LookupError('pop from empty TomboList')
    
    load = list.extend
    
    def loaded(self):
        return bool(self)
    
    def inspect(self):
        return tuple(sorted(self))

In [36]:
print(issubclass(TomboList, Tombola))
t = TomboList(range(100))
print(isinstance(t, Tombola))
print(TomboList.__mro__)

True
True
(<class '__main__.TomboList'>, <class 'list'>, <class 'object'>)


##### 11.9  Python使用register的方式